# load data from pkl file 

# print the number of "spurious examples" extracted

# print the sentence associated

# load the id of extracted examples and show if they are in an other pkl file

In [11]:
!pip install pandas

  Using cached pandas-2.2.3-cp311-cp311-win_amd64.whl.metadata (19 kB)
  Using cached pytz-2025.1-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.1-py2.py3-none-any.whl.metadata (1.4 kB)
Using cached pandas-2.2.3-cp311-cp311-win_amd64.whl (11.6 MB)
Using cached pytz-2025.1-py2.py3-none-any.whl (507 kB)
Using cached tzdata-2025.1-py2.py3-none-any.whl (346 kB)


In [18]:
import pickle
pkl_file = ['pkl_files/forget_examples.pkl', 'pkl_files/important_examples.pkl', 'pkl_files/important_examples_lid.pkl']
# Charger les exemples extraits
for file in pkl_file:
    with open(file, 'rb') as file:
        extracted_examples = pickle.load(file)

    # Vérifier le type de la structure de données
    print(f"Type de la structure chargée : {type(extracted_examples)}\n")

    # good one
    if isinstance(extracted_examples, dict):
        # Si c'est un dictionnaire
        for i, (key, value) in enumerate(extracted_examples.items()):
            if i >= 5:
                break
            print(f"Clé : {key}\nValeur : {value}\n")
            
    print(len(value))


Type de la structure chargée : <class 'dict'>

Clé : selected_examples
Valeur : [     4     33     34 ... 392665 392674 392691]

Clé : forgettables_b
Valeur : [     4     33     64 ... 376884 376897 376898]

30450
Type de la structure chargée : <class 'dict'>

Clé : selected_examples
Valeur : [     3      4     61 ... 392663 392664 392665]

Clé : forgettables_b
Valeur : [     3     61     63 ... 392401 392417 392418]

23391
Type de la structure chargée : <class 'dict'>

Clé : selected_examples
Valeur : [     3      4     61 ... 392663 392664 392665]

Clé : forgettables_b
Valeur : [     3     61     63 ... 391461 391526 391533]

23064


In [20]:
import pickle

# Liste des fichiers
pkl_files = [
            # 'pkl_files/forget_examples.pkl', 
            'pkl_files/important_examples_lid.pkl', 
            'pkl_files/important_examples_lid.pkl'
             ]

# Liste pour stocker les ensembles de valeurs
sets_of_values = []

# Charger les fichiers et extraire les valeurs
for file_path in pkl_files:
    with open(file_path, 'rb') as file:
        extracted_examples = pickle.load(file)
    
    # Combiner les valeurs des deux clés : selected_examples et forgettables_b
    combined_values = set()  # Initialiser comme un set pour éviter les doublons

    if isinstance(extracted_examples, dict):
        for key in ['selected_examples', 'forgettables_b']:
            if key in extracted_examples:
                combined_values.update(extracted_examples[key])  # Utiliser upd 
    
    # Convertir en ensemble pour faciliter les intersections
    sets_of_values.append(set(combined_values))

# Calculer l'intersection des trois ensembles
common_values = set.intersection(*sets_of_values)

# Afficher le résultat
print(f"Nombre de valeurs communes aux trois fichiers : {len(common_values)}")


Nombre de valeurs communes aux trois fichiers : 25946


In [23]:
import pickle

# Charger le fichier important_examples_lid.pkl
file_path = 'pkl_files/forget_examples.pkl'

with open(file_path, 'rb') as file:
    extracted_examples = pickle.load(file)

# Vérifier la structure des données pour comprendre où se trouvent les éléments
if isinstance(extracted_examples, dict):
    selected_examples_len = len(extracted_examples.get('selected_examples', []))
    forgettables_b_len = len(extracted_examples.get('forgettables_b', []))

    print(f"Nombre d'éléments dans 'selected_examples': {selected_examples_len}")
    print(f"Nombre d'éléments dans 'forgettables_b': {forgettables_b_len}")
    print(f"Total d'éléments combinés: {selected_examples_len + forgettables_b_len}")


Nombre d'éléments dans 'selected_examples': 36571
Nombre d'éléments dans 'forgettables_b': 30450
Total d'éléments combinés: 67021


In [ ]:
import pickle
import pandas as pd

# Charger les indices d'exemples sélectionnés
with open('mnli_bert_base/important_examples.pkl', 'rb') as file:
    extracted_examples = pickle.load(file)

selected_indices = extracted_examples['selected_examples']

# Charger le fichier train.tsv
# train_data = pd.read_csv('mnli/MNLI/MNLI/train.tsv', sep='\t')
train_data =  pd.read_csv('mnli/MNLI/MNLI/train.tsv',
                        delimiter='\t',
                        on_bad_lines='skip', 
                        skiprows=0,
                        quoting=3,
                        keep_default_na=False,
                        encoding="utf-8",)
# Vérifier la structure des colonnes
print(f"Colonnes disponibles : {train_data.columns.tolist()}")

extracted_data = []

for idx in selected_indices:  # Limité aux 5 premiers exemples
    row = train_data.iloc[idx]
    extracted_data.append({
        'Index': idx,
        'Sentence1': row['sentence1'],
        'Sentence2': row['sentence2'],
        'Gold_Label': row['gold_label']
    })

# Enregistrer les données extraites dans un fichier CSV
output_df = pd.DataFrame(extracted_data)
output_df.to_csv('pkl_files/extracted_important_examples.csv', index=False)

print("Les phrases extraites ont été enregistrées dans 'extracted_sentences.csv'.")


Colonnes disponibles : ['index', 'promptID', 'pairID', 'genre', 'sentence1_binary_parse', 'sentence2_binary_parse', 'sentence1_parse', 'sentence2_parse', 'sentence1', 'sentence2', 'label1', 'gold_label']
Les phrases extraites ont été enregistrées dans 'extracted_sentences.csv'.


In [17]:
import pickle
import pandas as pd

# Charger le dataset MNLI
mnli_data = pd.read_csv('mnli/MNLI/MNLI/train.tsv',
                        delimiter='\t',
                        on_bad_lines='skip', 
                        skiprows=0,
                        quoting=3,
                        keep_default_na=False,
                        encoding="utf-8",)

# Fonction pour analyser la distribution des labels
def analyze_label_distribution(pkl_file, method_name):
    with open(pkl_file, 'rb') as file:
        extracted_indices = pickle.load(file)
    
    # Filtrer les exemples extraits
    filtered_data = mnli_data[mnli_data['index'].isin(extracted_indices)]
    
    # Compter la distribution des labels
    label_counts = filtered_data['gold_label'].value_counts()
    print(f"Distribution des labels pour {method_name}:")
    print(label_counts)
    print('-' * 40)
    return label_counts

# Appliquer l'analyse pour chaque méthode
att_labels = analyze_label_distribution('pkl_files/important_examples.pkl', 'Attention Score')
lid_labels = analyze_label_distribution('pkl_files/important_examples_lid.pkl', 'LID Score')
forget_labels = analyze_label_distribution('pkl_files/forget_examples.pkl', 'Forget Events')

with open('pkl_files/forget_examples.pkl', 'rb') as file:
    extracted_data = pickle.load(file)

# Vérifier si c'est un dict
if isinstance(extracted_data, dict):
    extracted_indices = extracted_data['selected_examples']
else:
    extracted_indices = extracted_data

# Vérifier la colonne index
if 'index' in mnli_data.columns:
    filtered_data = mnli_data[mnli_data['index'].isin(extracted_indices)]
else:
    filtered_data = mnli_data.iloc[extracted_indices]

# Compter la distribution des labels
label_counts = filtered_data['gold_label'].value_counts()
print(label_counts)



Distribution des labels pour Attention Score:
Series([], Name: count, dtype: int64)
----------------------------------------
Distribution des labels pour LID Score:
Series([], Name: count, dtype: int64)
----------------------------------------
Distribution des labels pour Forget Events:
Series([], Name: count, dtype: int64)
----------------------------------------
gold_label
neutral          15881
entailment       10540
contradiction    10150
Name: count, dtype: int64


In [ ]:
import matplotlib.pyplot as plt

# Combiner les résultats dans un DataFrame
label_distribution = pd.DataFrame({
    'Attention Score': att_labels,
    'LID Score': lid_labels,
    'Forget Events': forget_labels
}).fillna(0)

# Plot
label_distribution.plot(kind='bar', figsize=(10, 6))
plt.title('Distribution des Labels par Méthode d’Extraction')
plt.xlabel('Classe (gold_label)')
plt.ylabel('Nombre d\'Exemples')
plt.xticks(rotation=0)
plt.show()


In [22]:
import pickle
import pandas as pd
from collections import Counter
import re

# Charger les indices d'exemples sélectionnés
with open('pkl_files/important_examples_lid.pkl', 'rb') as file:
    extracted_examples = pickle.load(file)

selected_indices = extracted_examples['selected_examples']

# Charger le fichier train.tsv
train_data = pd.read_csv(
    'mnli/MNLI/MNLI/train.tsv',
    delimiter='\t',
    on_bad_lines='skip', 
    quoting=3,
    keep_default_na=False,
    encoding="utf-8"
)

# Vérifier la structure des colonnes
print(f"Colonnes disponibles : {train_data.columns.tolist()}")

# Extraire les données et compter les mots
word_counter = Counter()

for idx in selected_indices:
    row = train_data.iloc[idx]
    sentences = f"{row['sentence1']} {row['sentence2']}"
    # Nettoyer et séparer les mots
    words = re.findall(r'\b\w+\b', sentences.lower())  # Conversion en minuscule et séparation des mots
    word_counter.update(words)

# Obtenir les 20 mots les plus fréquents
most_common_words = word_counter.most_common(50)

# Afficher les mots les plus fréquents
print("Mots les plus fréquents parmi les exemples spurious :")
for word, freq in most_common_words:
    print(f"{word}: {freq}")

# Enregistrer les mots fréquents dans un fichier CSV
output_df = pd.DataFrame(most_common_words, columns=['Word', 'Frequency'])
output_df.to_csv('pkl_files/most_common_words_lid.csv', index=False)

print("Les mots les plus fréquents ont été enregistrés dans 'most_common_words.csv'.")


Colonnes disponibles : ['index', 'promptID', 'pairID', 'genre', 'sentence1_binary_parse', 'sentence2_binary_parse', 'sentence1_parse', 'sentence2_parse', 'sentence1', 'sentence2', 'label1', 'gold_label']
Mots les plus fréquents parmi les exemples spurious :
the: 47519
to: 21329
of: 20034
and: 20021
a: 17375
in: 13304
i: 13300
that: 11605
it: 11027
is: 10261
you: 9368
s: 9306
for: 7059
was: 6574
they: 5620
are: 5522
t: 5300
on: 5036
have: 4863
not: 4711
be: 4645
with: 4541
as: 4152
he: 4077
but: 3936
we: 3737
know: 3544
there: 3486
uh: 3475
at: 3256
by: 3062
from: 3022
this: 2941
or: 2918
do: 2641
can: 2574
one: 2491
if: 2480
an: 2466
so: 2445
all: 2443
like: 2374
his: 2338
had: 2321
about: 2260
what: 2243
were: 2214
has: 2192
don: 2126
more: 2120
Les mots les plus fréquents ont été enregistrés dans 'most_common_words.csv'.


In [36]:
import pickle
import pandas as pd
from collections import Counter
import re

# Charger les indices d'exemples sélectionnés
with open('pkl_files/important_examples.pkl', 'rb') as file:
    extracted_examples = pickle.load(file)

selected_indices = extracted_examples['selected_examples']

# Charger le fichier train.tsv
train_data = pd.read_csv(
    'mnli/MNLI/MNLI/train.tsv',
    delimiter='\t',
    on_bad_lines='skip', 
    quoting=3,
    keep_default_na=False,
    encoding="utf-8"
)

# Vérifier la structure des colonnes
print(f"Colonnes disponibles : {train_data.columns.tolist()}")

# Compter la fréquence des mots dans l'ensemble du dataset
full_dataset_counter = Counter()
for _, row in train_data.iterrows():
    sentences = f"{row['sentence1']} {row['sentence2']}"
    words = re.findall(r'\b\w+\b', sentences.lower())
    full_dataset_counter.update(words)

# Extraire les données et compter les mots dans les exemples spurious
spurious_counter = Counter()
for idx in selected_indices:
    row = train_data.iloc[idx]
    sentences = f"{row['sentence1']} {row['sentence2']}"
    words = re.findall(r'\b\w+\b', sentences.lower())
    spurious_counter.update(words)

# Normaliser la fréquence par rapport à l'ensemble du dataset
normalized_freq = {
    word: (spurious_count / full_dataset_counter[word])
    for word, spurious_count in spurious_counter.items()
    if full_dataset_counter[word] > 0 and spurious_count > 500  # Considérer uniquement les mots avec freq > 5
}

sorted_words = sorted(normalized_freq.items(), key=lambda x: x[1], reverse=True)[:50]

# Afficher les mots les plus fréquents avec les deux fréquences
for word, norm_freq in sorted_words:
    freq = spurious_counter.get(word, 0)

# Enregistrer les mots fréquents avec les deux fréquences dans un fichier CSV
output_df = pd.DataFrame(
    [(word, spurious_counter.get(word, 0), norm_freq) for word, norm_freq in sorted_words],
    columns=['Word', 'Spurious_Frequency', 'Normalized_Frequency']
)
output_df.to_csv('pkl_files/most_common_words_important_20.csv', index=False)

print("Les mots les plus fréquents avec les fréquences ont été enregistrés.")



Colonnes disponibles : ['index', 'promptID', 'pairID', 'genre', 'sentence1_binary_parse', 'sentence2_binary_parse', 'sentence1_parse', 'sentence2_parse', 'sentence1', 'sentence2', 'label1', 'gold_label']
Les mots les plus fréquents avec les fréquences ont été enregistrés.


In [3]:
import pickle
import pandas as pd
from collections import Counter
import re

# Charger les indices d'exemples sélectionnés
with open('pkl_files/important_examples_lid.pkl', 'rb') as file:
    extracted_examples = pickle.load(file)

selected_indices = extracted_examples['selected_examples']

# Charger le fichier train.tsv
train_data = pd.read_csv(
    'mnli/MNLI/MNLI/train.tsv',
    delimiter='\t',
    on_bad_lines='skip', 
    quoting=3,
    keep_default_na=False,
    encoding="utf-8"
)

# Vérifier la structure des colonnes
print(f"Colonnes disponibles : {train_data.columns.tolist()}")

# Compter la fréquence des mots dans l'ensemble du dataset
full_dataset_counter = Counter()
total_words_in_dataset = 0
for _, row in train_data.iterrows():
    sentences = f"{row['sentence1']} {row['sentence2']}"
    words = re.findall(r'\b\w+\b', sentences.lower())
    full_dataset_counter.update(words)
    total_words_in_dataset += len(words)

# Extraire les données et compter les mots dans les exemples spurious
spurious_counter = Counter()
total_words_in_spurious = 0
for idx in selected_indices:
    row = train_data.iloc[idx]
    sentences = f"{row['sentence1']} {row['sentence2']}"
    words = re.findall(r'\b\w+\b', sentences.lower())
    spurious_counter.update(words)
    total_words_in_spurious += len(words)

# Normaliser la fréquence par rapport à l'ensemble du dataset
normalized_freq = {
    word: ( (spurious_count / total_words_in_spurious) / (full_dataset_counter[word] / total_words_in_dataset) )
    for word, spurious_count in spurious_counter.items()
    if full_dataset_counter[word] > 0 and spurious_count > 1  # Considérer uniquement les mots avec freq > 5
}

# Définir les seuils pour identifier un key pattern
SPURIOUS_FREQ_THRESHOLD = 10
NORMALIZED_FREQ_THRESHOLD = 2

# Identifier les key patterns
key_patterns = {
    word: (spurious_counter[word] > SPURIOUS_FREQ_THRESHOLD and normalized_freq[word] > NORMALIZED_FREQ_THRESHOLD)
    for word in normalized_freq
}

# Trier les mots par fréquence normalisée décroissante
sorted_words = sorted(normalized_freq.items(), key=lambda x: x[1], reverse=True)

for word, norm_freq in sorted_words:
    freq = spurious_counter.get(word, 0)
    is_key_pattern = key_patterns.get(word, False)
    print(f"{word}: Fréquence dans spurious = {freq}, Fréquence normalisée = {norm_freq:.4f}, Key Pattern = {is_key_pattern}")

# Enregistrer uniquement les key patterns dans un fichier CSV
key_pattern_words = [
    (word, spurious_counter.get(word, 0), norm_freq, True)
    for word, norm_freq in sorted_words
    if key_patterns.get(word, False)
]

output_df = pd.DataFrame(
    key_pattern_words,
    columns=['Word', 'Spurious_Frequency', 'Normalized_Frequency', 'Is_Key_Pattern']
)
output_df.to_csv('pkl_files/new_lid.csv', index=False)

print("Les mots les plus fréquents avec les fréquences et l'indication des key patterns ont été enregistrés dans 'most_common_spurious_words.csv'.")


Colonnes disponibles : ['index', 'promptID', 'pairID', 'genre', 'sentence1_binary_parse', 'sentence2_binary_parse', 'sentence1_parse', 'sentence2_parse', 'sentence1', 'sentence2', 'label1', 'gold_label']
bunts: Fréquence dans spurious = 6, Fréquence normalisée = 14.8014, Key Pattern = False
apb: Fréquence dans spurious = 3, Fréquence normalisée = 14.8014, Key Pattern = False
arb: Fréquence dans spurious = 3, Fréquence normalisée = 14.8014, Key Pattern = False
fas: Fréquence dans spurious = 3, Fréquence normalisée = 14.8014, Key Pattern = False
interpertation: Fréquence dans spurious = 3, Fréquence normalisée = 14.8014, Key Pattern = False
gunned: Fréquence dans spurious = 2, Fréquence normalisée = 14.8014, Key Pattern = False
unabated: Fréquence dans spurious = 2, Fréquence normalisée = 14.8014, Key Pattern = False
ayasofya: Fréquence dans spurious = 5, Fréquence normalisée = 14.8014, Key Pattern = False
incontinent: Fréquence dans spurious = 2, Fréquence normalisée = 14.8014, Key Patt

In [6]:
import pandas as pd

# Charger le fichier MNLI
file_path = 'train.tsv'  # Remplace par le chemin correct si nécessaire

# Lire le fichier TSV en ignorant les lignes mal formatées
df = pd.read_csv(
    'mnli/MNLI/MNLI/train.tsv',
    delimiter='\t',
    on_bad_lines='skip', 
    quoting=3,
    keep_default_na=False,
    encoding="utf-8"
)
# Compter les occurrences de chaque label
label_counts = df['gold_label'].value_counts()

# Afficher les résultats
for label in ['neutral', 'contradiction', 'entailment']:
    print(f"{label}: {label_counts.get(label, 0)}")


neutral: 130900
contradiction: 130903
entailment: 130899


In [7]:
130900+130900+130899

392699

In [14]:
26000/392699

0.06620847010050955